# Cross Validation in `Python`

One of the main goals of the statistical and machine learning (ML) models is to predict a response variable (in many cases this is the unique goal).

In this article we are going to study different algorithms oriented to measure the predictive power of a statistic or ML model.

## Train data-set

## Test data-set